In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (512, 512))
            images.append(img)
            labels.append(label)
    return images, labels

def main():
    data_dir = 'C:\\Users\\WERKIT ZAMBIA\\Downloads\\TB_Chest_Radiography_Database'

    # Load images
    normal_images, normal_labels = load_images_from_folder(os.path.join(data_dir, 'Normal'), 0)
    tb_images, tb_labels = load_images_from_folder(os.path.join(data_dir, 'Tuberculosis'), 1)

    # Combine and convert to numpy arrays
    all_images = np.array(normal_images + tb_images)
    all_labels = np.array(normal_labels + tb_labels)

    # Normalize images
    all_images = all_images / 255.0

    # Split the dataset
    X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=0.2, random_state=42)

    # Define the CNN model
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 3)),
        MaxPooling2D(2, 2),
        # Additional layers
        Flatten(),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, epochs=10, validation_split=0.1)

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {accuracy*100:.2f}%")

    # Save the model
    model.save('tb_detection_model.h5')

if __name__ == '__main__':
    main()


MemoryError: Unable to allocate 24.6 GiB for an array with shape (4200, 512, 512, 3) and data type float64

In [2]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def main():
    data_dir = 'C:\\Users\\WERKIT ZAMBIA\\Downloads\\TB_Chest_Radiography_Database'

    # Create ImageDataGenerators for training and validation
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2  # using 20% of the data for validation
    )

    train_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(256, 256),  # reducing image size
        batch_size=32,
        class_mode='binary',
        subset='training'
    )

    validation_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(256, 256),
        batch_size=32,
        class_mode='binary',
        subset='validation'
    )

    # Define the CNN model
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
        MaxPooling2D(2, 2),
        # Additional layers
        Flatten(),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // validation_generator.batch_size,
        epochs=10
    )

    # Save the model
    model.save('tb_detection_model.h5')

if __name__ == '__main__':
    main()


Found 3360 images belonging to 2 classes.
Found 840 images belonging to 2 classes.


ResourceExhaustedError: {{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[516128,128] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:AddV2]

In [2]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def main():
    data_dir = 'C:\\Users\\WERKIT ZAMBIA\\Downloads\\TB_Chest_Radiography_Database'

    # Create ImageDataGenerators for training and validation
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2  # using 20% of the data for validation
    )

    # Reduce batch size to decrease memory usage
    batch_size = 16

    train_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(256, 256),
        batch_size=batch_size,
        class_mode='binary',
        subset='training'
    )

    validation_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(256, 256),
        batch_size=batch_size,
        class_mode='binary',
        subset='validation'
    )

    # Define a simpler CNN model
    model = Sequential([
        Conv2D(16, (3, 3), activation='relu', input_shape=(256, 256, 3)),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['Recall'])

    # Train the model with reduced steps per epoch to further save memory
    model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size // 2,  # Half the steps
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size // 2,  # Half the steps
        epochs=10
    )

    # Save the model
    model.save('tb_detection_model.h5')

if __name__ == '__main__':
    main()


Found 3360 images belonging to 2 classes.
Found 840 images belonging to 2 classes.
Epoch 1/10


ValueError: in user code:

    File "c:\Users\WERKIT ZAMBIA\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\WERKIT ZAMBIA\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\WERKIT ZAMBIA\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\WERKIT ZAMBIA\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1028, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "c:\Users\WERKIT ZAMBIA\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1122, in compute_metrics
        self.compil

In [4]:
import cv2
import numpy as np
import tensorflow as tf

def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (512, 512))
    img = img / 255.0
    return img

def main():
    # Load the trained model
    model = tf.keras.models.load_model('tb_detection_model.h5')

    # Path to the new image for analysis
    image_path = input("Enter the path of the image to analyze: ")

    # Preprocess the image
    img = load_and_preprocess_image(image_path)
    img = np.expand_dims(img, axis=0)  # Expand dims to fit model input

    # Predict
    prediction = model.predict(img)
    result = 'TB Detected' if prediction[0][0] > 0.5 else 'No TB'
    print(result)

if __name__ == '__main__':
    main()


In [1]:
!pip install flask


In [2]:
from flask import Flask, request, render_template
import tensorflow as tf
import numpy as np
import cv2
import os

app = Flask(__name__)

# Load the trained model
model = tf.keras.models.load_model('tb_detection_model.h5')

def load_and_preprocess_image(image):
    img = cv2.imdecode(np.fromstring(image.read(), np.uint8), cv2.IMREAD_UNCHANGED)
    img = cv2.resize(img, (256, 256))  # Ensure this matches the training input
    img = img / 255.0
    return img

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        file = request.files['file']
        if file:
            img = load_and_preprocess_image(file)
            img = np.expand_dims(img, axis=0)  # Expand dims to fit model input
            prediction = model.predict(img)
            result = 'TB Detected' if prediction[0][0] > 0.5 else 'No TB'
            return result
    return '''
    <!doctype html>
    <title>Upload an image</title>
    <h1>Upload a Chest X-Ray Image for TB Analysis</h1>
    <form method=post enctype=multipart/form-data>
      <input type=file name=file>
      <input type=submit value=Analyze>
    </form>
    '''

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\WERKIT ZAMBIA\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
from flask import Flask, render_template, request
import tensorflow as tf
import numpy as np
import cv2
import mysql.connector
from datetime import datetime

app = Flask(__name__)

# Load the trained model
model = tf.keras.models.load_model('tb_detection_model.h5')

# Database configuration
db_config = {
    'user': 'root',
    'password': '',
    'host': 'localhost',
    'database': 'tb',
    'raise_on_warnings': True
}

def load_and_preprocess_image(image):
    img = cv2.imdecode(np.fromstring(image.read(), np.uint8), cv2.IMREAD_COLOR)
    img = cv2.resize(img, (256, 256))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
    img = img / 255.0
    return img

def save_result(name, id_number, nrc, date_of_testing, result):
    conn = None
    try:
        conn = mysql.connector.connect(**db_config)
        cursor = conn.cursor()
        query = "INSERT INTO users (name, id_number, nrc, date_of_testing, results) VALUES (%s, %s, %s, %s, %s)"
        cursor.execute(query, (name, id_number, nrc, date_of_testing, result))
        conn.commit()
    except mysql.connector.Error as err:
        print(f"Error: {err}")
    finally:
        if conn is not None and conn.is_connected():
            cursor.close()
            conn.close()

def get_user_details(id_number):
    conn = None
    try:
        conn = mysql.connector.connect(**db_config)
        cursor = conn.cursor()
        query = "SELECT * FROM users WHERE id_number = %s"
        cursor.execute(query, (id_number,))
        result = cursor.fetchone()
        return result
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if conn is not None and conn.is_connected():
            cursor.close()
            conn.close()

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    user_details = None
    search_id = ""
    user_not_found = False
    analysis_result = None  # Variable to store the analysis result

    if request.method == 'POST':
        if 'file' in request.files:
            file = request.files['file']
            if file:
                img = load_and_preprocess_image(file)
                img = np.expand_dims(img, axis=0)  # Expand dims to fit model input
                prediction = model.predict(img)
                analysis_result = 'TB Detected' if prediction[0][0] > 0.5 else 'No TB'
                
                
                 # Display result and user details form
                return '''
                    <h1>Result: {}</h1>
                    <form method="post">
                        <label for="name">Name:</label>
                        <input type="text" name="name" required>
                        <label for="id_number">ID Number:</label>
                        <input type="text" name="id_number" required>
                        <label for="nrc">NRC:</label>
                        <input type="text" name="nrc" required>
                        <label for="date_of_testing">Date of Testing:</label>
                        <input type="date" name="date_of_testing" required>
                        <input type="hidden" name="result" value="{}">
                        <input type="submit" value="Upload Results">
                    </form>
                '''.format(result, result)
        else:
            # Save user details and result to database
            name = request.form['name']
            id_number = request.form['id_number']
            nrc = request.form['nrc']
            date_of_testing = request.form['date_of_testing']
            result = request.form['result']
            save_result(name, id_number, nrc, date_of_testing, result)
            return '<h1>Results Uploaded Successfully</h1>'
                
                
                
                
                # Additional logic for saving result if needed
        elif 'search_id' in request.form:
            search_id = request.form.get('search_id')
            user_details = get_user_details(search_id)
            if not user_details:
                user_not_found = True  # User not found

        elif 'name' in request.form:
            # Save user details and result to database
            name = request.form['name']
            id_number = request.form['id_number']
            nrc = request.form['nrc']
            date_of_testing = request.form['date_of_testing']
            result = request.form['result']
            save_result(name, id_number, nrc, date_of_testing, result)
            return '<h1>Results Uploaded Successfully</h1>'

    return render_template('template.html', 
                           user_details=user_details,
                           search_id=search_id,
                           user_not_found=user_not_found,
                           analysis_result=analysis_result)

if __name__ == '__main__':
    app.run(debug=True)